## requirements.txt

In [ ]:
%%capture

!pip install sentencepiece
!pip install youtube-transcript-api
!pip install gradio

### sumarizer.py

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def Summarizer(link="Video", model="Pegasus", text="Text"):
  if model == "Pegasus":
      checkpoint = "google/pegasus-large"
  elif model == "mT5":
      checkpoint = "csebuetnlp/mT5_multilingual_XLSum"
  elif model == "BART":
      checkpoint = "sshleifer/distilbart-cnn-12-6"

  tokenizer = AutoTokenizer.from_pretrained(checkpoint)
  model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

  if link == "Video":
    video_id = link.split("=")[1]
    transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=["pt", "en"])
    FinalTranscript = ' '.join([i['text'] for i in transcript])

  else:
    FinalTranscript = text

  inputs = tokenizer(FinalTranscript,
                    max_length=1024,
                    truncation=True,
                    return_tensors="pt")

  summary_ids = model.generate(inputs["input_ids"])
  summary = tokenizer.batch_decode(summary_ids,
                                  skip_special_tokens=True,
                                  clean_up_tokenization_spaces=False)


  return summary[0]


## App.py

In [ ]:
import gradio as gr

interface = gr.Interface(
    Summarizer,
     [
         gr.Textbox(), gr.Textbox(), gr.Textbox()
     ],
    outputs="text"
    )


#interface.launch(debug=True)